In [16]:
from FileReader import FileReader

In [2]:
classInstance = FileReader
df_list = classInstance.get_dataframe() #IMPORTANT:  saves a pickle to output/simple or output/full. 

#The pickle is the same as df.



 553480082996879360

 581153923987206146

 500298588992593920

 552788945017516032

 524948206023880704

 553561170637238272

 758159624122097664

 767725956706414592

 580352273001410560

 581359544682614784

 763098277986209792

 581290271997968384

 775057555865206784

 500280249629036544

 498293668655423488

 774991078265094144

 553553288625672192

 524961721744900097

 524941720249978880

 544294893146091520

 768859780240773121

 764927075522260992

 544274544174071809

 498486826269548545

 544315472075042818

 524923293711998976

 769988636754505729

 524936793633083394

 553480082996879360

 581153923987206146

 500298588992593920

 552788945017516032

 524948206023880704

 553561170637238272

 758159624122097664

 767725956706414592

 580352273001410560

 581359544682614784

 763098277986209792

 581290271997968384

 775057555865206784

 500280249629036544

 498293668655423488

 774991078265094144

 553553288625672192

 524961721744900097

 524941720249978880

 54429489314

# IMPORTANT NOTE:

Word embeddings are expressed as pickle files.
Reading of tweet data is converted to a Pandas Dataframe format and then finally into a pickle file, located output/simple or output/full.

It is important that you first execute file_reader.py and *then* this notebook, run_tests.ipynb.

In [17]:
import sys
sys.path.insert(1, "./feature-extraction/embed-extractor")
from EmbedExtractor import EmbedExtractor
sys.path.insert(1, "./feature-extraction/vulgar-extractor")
from VulgarExtractor import VulgarExtractor
sys.path.insert(1, "./feature-extraction/twitter-parser")
from TwitterParser import TwitterParser
import classifiers
import pandas as pd
import numpy as np
import json


In [18]:
ee = EmbedExtractor()

In [78]:
def normalize(column_name, df):
    std = df[column_name].std()
    norm_col = df[column_name].apply(lambda x: x - std)
    df[column_name] = norm_col

# builds the labels and vectorizations of given data
#if you want to fool around with including/excluding certain features and whatnot, this is the place to do it

def labels_and_vectors(file, index=0):
    df = pd.read_pickle(file)
    
    wordlist = VulgarExtractor.vulgarWords("feature-extraction/vulgar-extractor/badwords.txt") 
    dftext = df[['text']]
    result = dftext.applymap(lambda x: VulgarExtractor.containsVulgar(x,wordlist))
    df['isVulgar'] = result

    word_embeddings = [ee.tweetVec(tagged_line) for tagged_line in df['text']]
    textlist = [txt.replace('\n','') for txt in df['text'].tolist()]
    tagged_sents = TwitterParser.tag(textlist)
    df['POS'] = tagged_sents

    processed_sents = []
    for tagged_sent in df['POS']:
        processed_words = []
        for word, tag in tagged_sent:
            if tag == 'U':
                processed_words.append('someurl')
            elif tag == '@':
                processed_words.append('@someuser')
            else:
                processed_words.append(word)
        sent = ' '.join(processed_words)
        processed_sents.append(sent)
    df['text'] = processed_sents

    word_counts = [TwitterParser.word_count(tagged_line) for tagged_line in df['POS']]
    pos_count_list = [TwitterParser.pos_counts(tagged_line) for tagged_line in df['POS']]
    contains_adjs = [TwitterParser.contains_adjectives(tagged_line) for tagged_line in df['POS']]
    contains_urls = [TwitterParser.contains_url(tagged_line) for tagged_line in df['POS']]
    contains_emojis = [TwitterParser.contains_emoji(tagged_line) for tagged_line in df['POS']]
    contains_abbrevs = [TwitterParser.contains_abbreviation(tagged_line) for tagged_line in df['POS']]

    df['wordCount'] = word_counts
    df['posCounts'] = pos_count_list
    df['containsAdjective'] = contains_adjs
    df['containsURL'] = contains_urls
    df['containsEmoji'] = contains_emojis
    df['containsAbbreviation'] = contains_abbrevs
    df['wordEmbedding'] = word_embeddings

    
    for i, tag in enumerate(TwitterParser.tagset):
        tag_counts = []
        for pos_counts in df['posCounts']:
            tag_counts.append(pos_counts[i])
        column_name = 'num_' + tag
        df[column_name] = tag_counts
        normalize(column_name, df)
        
    # Changes "true"/"false"/"unverified" to numeric values, just like the in the early cells
    df.loc[df.classification == 'true', 'classification'] = 1
    df.loc[df.classification == 'false', 'classification'] = 0
    df.loc[df.classification == 'unverified', 'classification'] = 2
    # getting the labels

    #removed containsURL
    attributes = ['isVulgar', 'containsAdjective', 'containsURL', 'containsEmoji', 'containsAbbreviation', 'wordCount']
    for tag in TwitterParser.tagset:
        attributes.append('num_' + tag)

    labels = df['classification']
    labels = [l for l in labels]
    labels = np.array(labels)


    # getting the values as a list of lists
    values = df[attributes].values.tolist()
    word_embedding_values = df['wordEmbedding'].values.tolist()


#     #Below puts the tweet ID as a feature. Comment this out if you aren't using tweetID
#     for i,index in enumerate(df.index):
#        dev_values[i].append(int(index))


#UNCOMMENT THIS IN ORDER TO INCOPORATE WORD_EMBEDDINGS AGAIN
    for i,d in enumerate(word_embedding_values):
       values[i].extend(d)

    values = np.array(values)
    if index == 1:
        return df.index, values
    

    
    return labels, values
    

In [79]:
# note that the indices of labels-to-values should not be mismatched
tr_labels, tr_values = labels_and_vectors('output/simple/train_data_simple.pickle')
indices, dev_values,= labels_and_vectors('output/simple/goldtest_data_simple.pickle', index=1)



In [81]:
#change classifier here
predictions, probabilities = classifiers.naive_bayes(tr_values, tr_labels, dev_values)
ps = []

# need to convert the numerical predictions back into their string values
for i, p in enumerate(predictions):
    if p == 0:
        ps.append('false')
    if p == 1:
        ps.append('true')
    if p == 2:
        ps.append('unverified')

# creates pairings of the prediction and the probability of the prediction
pred_probs_pairs = [[ps[i], probabilities[i][predictions[i]]] for i in range(len(predictions))] 
#attaches the tweetID (called reference_id in the score.py file)
pred_dict = {index:pred_probs_pairs[i] for i,index in enumerate(indices)}

In [82]:
with open('output/classifier_output/goldtest_nb.json', 'w') as outfile:
    json.dump(pred_dict, outfile)

In [84]:
!python3 scorer/score.py data/semeval2017-task8-dataset/goldtest/subtaskb.json output/classifier_output/goldtest_nb.json

28 entries in reference file
matching entry: 775057555865206784
matching entry: 498486826269548545
matching entry: 774991078265094144
matching entry: 524948206023880704
matching entry: 769988636754505729
matching entry: 544315472075042818
matching entry: 524936793633083394
matching entry: 553561170637238272
matching entry: 763098277986209792
matching entry: 498293668655423488
matching entry: 524941720249978880
matching entry: 553553288625672192
matching entry: 764927075522260992
matching entry: 544274544174071809
matching entry: 758159624122097664
matching entry: 552788945017516032
matching entry: 500280249629036544
matching entry: 580352273001410560
matching entry: 500298588992593920
matching entry: 524923293711998976
matching entry: 553480082996879360
matching entry: 544294893146091520
matching entry: 767725956706414592
matching entry: 581153923987206146
matching entry: 581359544682614784
matching entry: 524961721744900097
matching entry: 768859780240773121
matching entry: 5812902719